In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from avaml.aggregatedata import ForecastDataset, LabeledData, REG_ENG, CsvMissingError

# Read in data

In [2]:
model_prefix = ''
days = 1
regobs_types = list(REG_ENG.keys())
labeled_data = None
try:
    print("Reading csv")
    labeled_data = LabeledData.from_csv(days=days, regobs_types=regobs_types, with_varsom=True)
except CsvMissingError:
    print("Csv missing. Fetching online data. (This takes a long time.)")
    labeled_data = ForecastDataset(regobs_types=regobs_types).label(days=days, with_varsom=True)
    labeled_data.to_csv()

Reading csv


`download.py` should have a flag where `main_text` is included instead of excluded. I might just create a separate structure for it.

https://github.com/NVE/avalanche_ml/blob/feature/meta-model/avaml/download.py

Use method `_get_raw_varsom()` to get a list of objects (only use year, not date and days) to get a list of data attributes. Text data should be its own attribute.

# EDA

In [3]:
labeled_data

In [4]:
vars(labeled_data).keys()

dict_keys(['data', 'row_weight', 'label', 'pred', 'days', 'with_varsom', 'regobs_types', 'single', 'seasons'])

In [5]:
vars(labeled_data)

{'data':                 region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                              0              0              0              0   
 2017-12-07 3003            0.0            0.0            1.0            0.0   
            3007            0.0            0.0            0.0            0.0   
            3009            0.0            0.0            0.0            0.0   
            3010            0.0            0.0            0.0            0.0   
            3011            0.0            0.0            0.0            0.0   
 ...                        ...            ...            ...            ...   
 2021-01-12 3031            0.0            0.0            0.0            0.0   
            3032            0.0            0.0            0.0            0.0   
            3034            0.0            0.0            0.0            0.0   
            3035            0.0            0.0            0.0            0.0   
            3037            0.0 

In [6]:
labeled_data.label

CLASS                                            \
                                                                         
                danger_level emergency_warning   problem_1   problem_2   
2017-12-07 3003            2         Ikke gitt  drift-slab               
           3007            2         Ikke gitt    pwl-slab  drift-slab   
           3009            2         Ikke gitt    pwl-slab  drift-slab   
           3010            3         Ikke gitt  drift-slab    pwl-slab   
           3011            3         Ikke gitt  drift-slab    pwl-slab   
...                      ...               ...         ...         ...   
2021-01-12 3031            3         Ikke gitt    pwl-slab               
           3032            3         Ikke gitt    pwl-slab               
           3034            3         Ikke gitt    pwl-slab               
           3035            3         Ikke gitt    pwl-slab               
           3037            3         Ikke gitt    pwl-slab               

                                                                         ...  \
                                         drift-slab                      ...   
                problem_3 problem_amount      cause dist dsize lev_fill  ...   
2017-12-07 3003                        1  snowdrift    1     2        1  ...   
           3007                        2   new-snow    2     2        1  ...   
           3009                        2   new-snow    2     2        1  ...   
           3010                        2   new-snow    3     2        1  ...   
           3011                        2   new-snow    3     2        1  ...   
...                   ...            ...        ...  ...   ...      ...  ...   
2021-01-12 3031                        1          0    0     0        0  ...   
           3032                        1          0    0     0        0  ...   
           3034                        1          0    0     0        0  ...   
           3035                        1          0    0     0        0  ...   
           3037                        1          0    0     0        0  ...   

                     REAL                                                      \
                new-loose         new-slab         pwl-slab         wet-loose   
                  lev_max lev_min  lev_max lev_min  lev_max lev_min   lev_max   
2017-12-07 3003       0.0     0.0      0.0     0.0      0.0     0.0       0.0   
           3007       0.0     0.0      0.0     0.0    700.0   200.0       0.0   
           3009       0.0     0.0      0.0     0.0    700.0   200.0       0.0   
           3010       0.0     0.0      0.0     0.0    700.0   200.0       0.0   
           3011       0.0     0.0      0.0     0.0    700.0   200.0       0.0   
...                   ...     ...      ...     ...      ...     ...       ...   
2021-01-12 3031       0.0     0.0      0.0     0.0    600.0   600.0       0.0   
           3032       0.0     0.0      0.0     0.0    800.0   800.0       0.0   
           3034       0.0     0.0      0.0     0.0    600.0   600.0       0.0   
           3035       0.0     0.0      0.0     0.0    800.0   800.0       0.0   
           3037       0.0     0.0      0.0     0.0    800.0   800.0       0.0   

                                          
                        wet-slab          
                lev_min  lev_max lev_min  
2017-12-07 3003     0.0      0.0     0.0  
           3007     0.0      0.0     0.0  
           3009     0.0      0.0     0.0  
           3010     0.0      0.0     0.0  
           3011     0.0      0.0     0.0  
...                 ...      ...     ...  
2021-01-12 3031     0.0      0.0     0.0  
           3032     0.0      0.0     0.0  
           3034     0.0      0.0     0.0  
           3035     0.0      0.0     0.0  
           3037     0.0      0.0     0.0  

[12651 rows x 69 columns]

In [7]:
labeled_data.data.head()

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                             0              0              0              0   
2017-12-07 3003            0.0            0.0            1.0            0.0   
           3007            0.0            0.0            0.0            0.0   
           3009            0.0            0.0            0.0            0.0   
           3010            0.0            0.0            0.0            0.0   
           3011            0.0            0.0            0.0            0.0   

                region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                             0              0              0              0   
2017-12-07 3003            0.0            0.0            0.0            0.0   
           3007            0.0            0.0            1.0            0.0   
           3009            0.0            0.0            0.0            0.0   
           3010            0.0            0.0            0.0            0.0   
           3011            0.0            0.0            0.0            0.0   

                region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4  \
                             0              0  ...                          4   
2017-12-07 3003            0.0            0.0  ...                        0.0   
           3007            0.0            0.0  ...                        0.0   
           3009            1.0            0.0  ...                        0.0   
           3010            0.0            1.0  ...                        0.0   
           3011            0.0            0.0  ...                        0.0   

                               accuracy                           
                   5    6    7        2    3    4    5    6    7  
2017-12-07 3003  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
           3007  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
           3009  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
           3010  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
           3011  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 6562 columns]

In [8]:
labeled_data.data.shape

(12651, 6562)

In [9]:
labeled_data.data.index

MultiIndex([('2017-12-07', 3003),
            ('2017-12-07', 3007),
            ('2017-12-07', 3009),
            ('2017-12-07', 3010),
            ('2017-12-07', 3011),
            ('2017-12-07', 3012),
            ('2017-12-07', 3013),
            ('2017-12-07', 3014),
            ('2017-12-07', 3015),
            ('2017-12-07', 3016),
            ...
            ('2021-01-12', 3023),
            ('2021-01-12', 3024),
            ('2021-01-12', 3027),
            ('2021-01-12', 3028),
            ('2021-01-12', 3029),
            ('2021-01-12', 3031),
            ('2021-01-12', 3032),
            ('2021-01-12', 3034),
            ('2021-01-12', 3035),
            ('2021-01-12', 3037)],
           length=12651)

In [10]:
# get first level index, date
labeled_data.data.index.get_level_values(0)

Index(['2017-12-07', '2017-12-07', '2017-12-07', '2017-12-07', '2017-12-07',
       '2017-12-07', '2017-12-07', '2017-12-07', '2017-12-07', '2017-12-07',
       ...
       '2021-01-12', '2021-01-12', '2021-01-12', '2021-01-12', '2021-01-12',
       '2021-01-12', '2021-01-12', '2021-01-12', '2021-01-12', '2021-01-12'],
      dtype='object', length=12651)

In [11]:
# get second level index, region
labeled_data.data.index.get_level_values(1)

Int64Index([3003, 3007, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016,
            ...
            3023, 3024, 3027, 3028, 3029, 3031, 3032, 3034, 3035, 3037],
           dtype='int64', length=12651)

### Clustering data by region and time

I am interested in understanding how certain input variables are correlated to the output variables of interest. This includes how different snow pack conditions vary with avlanche problems and danger level warnings, but also how these input and output variables change with time, across different regions, etc.

The data is formatted as a MultiIndex in Pandas. This means we can first subset by time, then by region ID.

In [12]:
# look at df from one day
labeled_data.data.loc['2017-12-07'].shape

(21, 6562)

In [13]:
labeled_data.data.loc['2017-12-07']

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                  0              0              0              0   
3003            0.0            0.0            1.0            0.0   
3007            0.0            0.0            0.0            0.0   
3009            0.0            0.0            0.0            0.0   
3010            0.0            0.0            0.0            0.0   
3011            0.0            0.0            0.0            0.0   
3012            0.0            0.0            0.0            0.0   
3013            0.0            0.0            0.0            0.0   
3014            0.0            0.0            0.0            0.0   
3015            0.0            0.0            0.0            0.0   
3016            0.0            0.0            0.0            0.0   
3017            0.0            0.0            0.0            0.0   
3022            0.0            0.0            0.0            0.0   
3023            0.0            0.0            0.0            0.0   
3024            0.0            0.0            0.0            0.0   
3027            0.0            0.0            0.0            0.0   
3028            0.0            0.0            0.0            0.0   
3029            0.0            0.0            0.0            0.0   
3031            0.0            0.0            0.0            0.0   
3032            0.0            0.0            0.0            0.0   
3034            0.0            0.0            0.0            0.0   
3035            0.0            0.0            0.0            0.0   

     region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                  0              0              0              0   
3003            0.0            0.0            0.0            0.0   
3007            0.0            0.0            1.0            0.0   
3009            0.0            0.0            0.0            0.0   
3010            0.0            0.0            0.0            0.0   
3011            0.0            0.0            0.0            0.0   
3012            0.0            0.0            0.0            0.0   
3013            0.0            0.0            0.0            0.0   
3014            0.0            0.0            0.0            0.0   
3015            0.0            0.0            0.0            0.0   
3016            0.0            0.0            0.0            0.0   
3017            0.0            0.0            0.0            0.0   
3022            0.0            0.0            0.0            0.0   
3023            0.0            0.0            0.0            0.0   
3024            0.0            0.0            0.0            0.0   
3027            0.0            0.0            0.0            0.0   
3028            0.0            0.0            0.0            0.0   
3029            0.0            0.0            0.0            0.0   
3031            0.0            0.0            0.0            0.0   
3032            0.0            0.0            0.0            0.0   
3034            0.0            0.0            0.0            0.0   
3035            0.0            0.0            0.0            0.0   

     region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4            \
                  0              0  ...                          4    5    6   
3003            0.0            0.0  ...                        0.0  0.0  0.0   
3007            0.0            0.0  ...                        0.0  0.0  0.0   
3009            1.0            0.0  ...                        0.0  0.0  0.0   
3010            0.0            1.0  ...                        0.0  0.0  0.0   
3011            0.0            0.0  ...                        0.0  0.0  0.0   
3012            0.0            0.0  ...                        0.0  0.0  0.0   
3013            0.0            0.0  ...                        0.0  0.0  0.0   
3014            0.0            0.0  ...                        0.0  0.0  0.0   
3015            0.0            0.0  ...                        0.0  0.0  0.

In [14]:
# now subset by day and region
labeled_data.data.loc['2017-12-07', 3003]

region_id_3001  0    0.0
region_id_3002  0    0.0
region_id_3003  0    1.0
region_id_3004  0    0.0
region_id_3005  0    0.0
                    ... 
accuracy        3    0.0
                4    0.0
                5    0.0
                6    0.0
                7    0.0
Name: (2017-12-07, 3003), Length: 6562, dtype: float64

**Does the `LabeledData` class have a method that will drop extra region columns (`region_id_3001`, for example) and just get the columns of that pertain to the region of interest?**

There is a method, but it is not in `master` yet, Aron is sending a link.

In [15]:
# first, rename indices
labeled_data.data = labeled_data.data.rename_axis(['date','region'])

In [16]:
labeled_data.data

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                               0              0              0              0   
date       region                                                               
2017-12-07 3003              0.0            0.0            1.0            0.0   
           3007              0.0            0.0            0.0            0.0   
           3009              0.0            0.0            0.0            0.0   
           3010              0.0            0.0            0.0            0.0   
           3011              0.0            0.0            0.0            0.0   
...                          ...            ...            ...            ...   
2021-01-12 3031              0.0            0.0            0.0            0.0   
           3032              0.0            0.0            0.0            0.0   
           3034              0.0            0.0            0.0            0.0   
           3035              0.0            0.0            0.0            0.0   
           3037              0.0            0.0            0.0            0.0   

                  region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                               0              0              0              0   
date       region                                                               
2017-12-07 3003              0.0            0.0            0.0            0.0   
           3007              0.0            0.0            1.0            0.0   
           3009              0.0            0.0            0.0            0.0   
           3010              0.0            0.0            0.0            0.0   
           3011              0.0            0.0            0.0            0.0   
...                          ...            ...            ...            ...   
2021-01-12 3031              0.0            0.0            0.0            0.0   
           3032              0.0            0.0            0.0            0.0   
           3034              0.0            0.0            0.0            0.0   
           3035              0.0            0.0            0.0            0.0   
           3037              0.0            0.0            0.0            0.0   

                  region_id_3009 region_id_3010  ...  \
                               0              0  ...   
date       region                                ...   
2017-12-07 3003              0.0            0.0  ...   
           3007              0.0            0.0  ...   
           3009              1.0            0.0  ...   
           3010              0.0            1.0  ...   
           3011              0.0            0.0  ...   
...                          ...            ...  ...   
2021-01-12 3031              0.0            0.0  ...   
           3032              0.0            0.0  ...   
           3034              0.0            0.0  ...   
           3035              0.0            0.0  ...   
           3037              0.0            0.0  ...   

                  regobs_snowprofile_t_min_4                accuracy       \
                                           4    5    6    7        2    3   
date       region                                                           
2017-12-07 3003                          0.0  0.0  0.0  0.0      0.0  0.0   
           3007                          0.0  0.0  0.0  0.0      0.0  0.0   
           3009                          0.0  0.0  0.0  0.0      0.0  0.0   
           3010                          0.0  0.0  0.0  0.0      0.0  0.0   
           3011                          0.0  0.0  0.0  0.0      0.0  0.0   
...                                      ...  ...  ...  ...      ...  ...   
2021-01-12 3031                          0.0  0.0  0.0  0.0      0.0  0.0   
           3032                          0.0  0.0  0.0  0.0      0.0  0.0   
           3034                          0.0  0.0  0.0  0.0      0.0  0.0   
           3035                        

To group by regions, we first need to reorder the indices of the MultiIndex object.

In [16]:
regions = labeled_data.data.reorder_levels([1, 0])
regions

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                             0              0              0              0   
3003 2017-12-07            0.0            0.0            1.0            0.0   
3007 2017-12-07            0.0            0.0            0.0            0.0   
3009 2017-12-07            0.0            0.0            0.0            0.0   
3010 2017-12-07            0.0            0.0            0.0            0.0   
3011 2017-12-07            0.0            0.0            0.0            0.0   
...                        ...            ...            ...            ...   
3031 2021-01-12            0.0            0.0            0.0            0.0   
3032 2021-01-12            0.0            0.0            0.0            0.0   
3034 2021-01-12            0.0            0.0            0.0            0.0   
3035 2021-01-12            0.0            0.0            0.0            0.0   
3037 2021-01-12            0.0            0.0            0.0            0.0   

                region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                             0              0              0              0   
3003 2017-12-07            0.0            0.0            0.0            0.0   
3007 2017-12-07            0.0            0.0            1.0            0.0   
3009 2017-12-07            0.0            0.0            0.0            0.0   
3010 2017-12-07            0.0            0.0            0.0            0.0   
3011 2017-12-07            0.0            0.0            0.0            0.0   
...                        ...            ...            ...            ...   
3031 2021-01-12            0.0            0.0            0.0            0.0   
3032 2021-01-12            0.0            0.0            0.0            0.0   
3034 2021-01-12            0.0            0.0            0.0            0.0   
3035 2021-01-12            0.0            0.0            0.0            0.0   
3037 2021-01-12            0.0            0.0            0.0            0.0   

                region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4  \
                             0              0  ...                          4   
3003 2017-12-07            0.0            0.0  ...                        0.0   
3007 2017-12-07            0.0            0.0  ...                        0.0   
3009 2017-12-07            1.0            0.0  ...                        0.0   
3010 2017-12-07            0.0            1.0  ...                        0.0   
3011 2017-12-07            0.0            0.0  ...                        0.0   
...                        ...            ...  ...                        ...   
3031 2021-01-12            0.0            0.0  ...                        0.0   
3032 2021-01-12            0.0            0.0  ...                        0.0   
3034 2021-01-12            0.0            0.0  ...                        0.0   
3035 2021-01-12            0.0            0.0  ...                        0.0   
3037 2021-01-12            0.0            0.0  ...                        0.0   

                               accuracy                           
                   5    6    7        2    3    4    5    6    7  
3003 2017-12-07  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3007 2017-12-07  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3009 2017-12-07  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3010 2017-12-07  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3011 2017-12-07  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
...              ...  ...  ...      ...  ...  ...  ...  ...  ...  
3031 2021-01-12  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3032 2021-01-12  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3034 2021-01-12  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3035 2021-01-12  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
3037 2021-01-12  0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  

[12651 rows x 6562 columns]

In [17]:
regions.loc[3003]

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                        0              0              0              0   
2017-12-07            0.0            0.0            1.0            0.0   
2017-12-08            0.0            0.0            1.0            0.0   
2017-12-09            0.0            0.0            1.0            0.0   
2017-12-10            0.0            0.0            1.0            0.0   
2017-12-11            0.0            0.0            1.0            0.0   
...                   ...            ...            ...            ...   
2021-01-08            0.0            0.0            1.0            0.0   
2021-01-09            0.0            0.0            1.0            0.0   
2021-01-10            0.0            0.0            1.0            0.0   
2021-01-11            0.0            0.0            1.0            0.0   
2021-01-12            0.0            0.0            1.0            0.0   

           region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                        0              0              0              0   
2017-12-07            0.0            0.0            0.0            0.0   
2017-12-08            0.0            0.0            0.0            0.0   
2017-12-09            0.0            0.0            0.0            0.0   
2017-12-10            0.0            0.0            0.0            0.0   
2017-12-11            0.0            0.0            0.0            0.0   
...                   ...            ...            ...            ...   
2021-01-08            0.0            0.0            0.0            0.0   
2021-01-09            0.0            0.0            0.0            0.0   
2021-01-10            0.0            0.0            0.0            0.0   
2021-01-11            0.0            0.0            0.0            0.0   
2021-01-12            0.0            0.0            0.0            0.0   

           region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4       \
                        0              0  ...                          4    5   
2017-12-07            0.0            0.0  ...                        0.0  0.0   
2017-12-08            0.0            0.0  ...                        0.0  0.0   
2017-12-09            0.0            0.0  ...                        0.0  0.0   
2017-12-10            0.0            0.0  ...                        0.0  0.0   
2017-12-11            0.0            0.0  ...                        0.0  0.0   
...                   ...            ...  ...                        ...  ...   
2021-01-08            0.0            0.0  ...                        0.0  0.0   
2021-01-09            0.0            0.0  ...                        0.0  0.0   
2021-01-10            0.0            0.0  ...                        0.0  0.0   
2021-01-11            0.0            0.0  ...                        0.0  0.0   
2021-01-12            0.0            0.0  ...                        0.0  0.0   

                     accuracy                           
              6    7        2    3    4    5    6    7  
2017-12-07  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2017-12-08  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2017-12-09  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2017-12-10  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2017-12-11  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
...         ...  ...      ...  ...  ...  ...  ...  ...  
2021-01-08  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-09  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-10  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-11  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-12  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  

[568 rows x 6562 columns]

In [26]:
regions.loc[3003].tail()

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                        0              0              0              0   
2021-01-08            0.0            0.0            1.0            0.0   
2021-01-09            0.0            0.0            1.0            0.0   
2021-01-10            0.0            0.0            1.0            0.0   
2021-01-11            0.0            0.0            1.0            0.0   
2021-01-12            0.0            0.0            1.0            0.0   

           region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                        0              0              0              0   
2021-01-08            0.0            0.0            0.0            0.0   
2021-01-09            0.0            0.0            0.0            0.0   
2021-01-10            0.0            0.0            0.0            0.0   
2021-01-11            0.0            0.0            0.0            0.0   
2021-01-12            0.0            0.0            0.0            0.0   

           region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4       \
                        0              0  ...                          4    5   
2021-01-08            0.0            0.0  ...                        0.0  0.0   
2021-01-09            0.0            0.0  ...                        0.0  0.0   
2021-01-10            0.0            0.0  ...                        0.0  0.0   
2021-01-11            0.0            0.0  ...                        0.0  0.0   
2021-01-12            0.0            0.0  ...                        0.0  0.0   

                     accuracy                           
              6    7        2    3    4    5    6    7  
2021-01-08  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-09  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-10  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-11  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-12  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 6562 columns]

Now we could average over columns, sum, etc. We could try to count, as well, for histograms. We would need to use the method `drop_regions()` first to make this intelligible.

Could we embed a picture of Norway, and then map these averages or sums onto the avlanche regions? Are the avalanche regions in a format that GeoPandas can read in?

In [29]:
regions.groupby(level=0).sum()

region_id_3001 region_id_3002 region_id_3003 region_id_3004  \
                  0              0              0              0   
3003            0.0            0.0          568.0            0.0   
3006            0.0            0.0            0.0            0.0   
3007            0.0            0.0            0.0            0.0   
3009            0.0            0.0            0.0            0.0   
3010            0.0            0.0            0.0            0.0   
3011            0.0            0.0            0.0            0.0   
3012            0.0            0.0            0.0            0.0   
3013            0.0            0.0            0.0            0.0   
3014            0.0            0.0            0.0            0.0   
3015            0.0            0.0            0.0            0.0   
3016            0.0            0.0            0.0            0.0   
3017            0.0            0.0            0.0            0.0   
3022            0.0            0.0            0.0            0.0   
3023            0.0            0.0            0.0            0.0   
3024            0.0            0.0            0.0            0.0   
3027            0.0            0.0            0.0            0.0   
3028            0.0            0.0            0.0            0.0   
3029            0.0            0.0            0.0            0.0   
3031            0.0            0.0            0.0            0.0   
3032            0.0            0.0            0.0            0.0   
3034            0.0            0.0            0.0            0.0   
3035            0.0            0.0            0.0            0.0   
3037            0.0            0.0            0.0            0.0   

     region_id_3005 region_id_3006 region_id_3007 region_id_3008  \
                  0              0              0              0   
3003            0.0            0.0            0.0            0.0   
3006            0.0          161.0            0.0            0.0   
3007            0.0            0.0          580.0            0.0   
3009            0.0            0.0            0.0            0.0   
3010            0.0            0.0            0.0            0.0   
3011            0.0            0.0            0.0            0.0   
3012            0.0            0.0            0.0            0.0   
3013            0.0            0.0            0.0            0.0   
3014            0.0            0.0            0.0            0.0   
3015            0.0            0.0            0.0            0.0   
3016            0.0            0.0            0.0            0.0   
3017            0.0            0.0            0.0            0.0   
3022            0.0            0.0            0.0            0.0   
3023            0.0            0.0            0.0            0.0   
3024            0.0            0.0            0.0            0.0   
3027            0.0            0.0            0.0            0.0   
3028            0.0            0.0            0.0            0.0   
3029            0.0            0.0            0.0            0.0   
3031            0.0            0.0            0.0            0.0   
3032            0.0            0.0            0.0            0.0   
3034            0.0            0.0            0.0            0.0   
3035            0.0            0.0            0.0            0.0   
3037            0.0            0.0            0.0            0.0   

     region_id_3009 region_id_3010  ... regobs_snowprofile_t_min_4        \
                  0              0  ...                          4     5   
3003            0.0            0.0  ...                      -11.4 -11.4   
3006            0.0            0.0  ...                        0.0   0.0   
3007            0.0            0.0  ...                        0.0   0.0   
3009          580.0            0.0  ...                        0.0   0.0   
3010            0.0          580.0  ...                      -13.0 -13.0   
3011            0.0            0.0  ...                      -13.7 -13.

In [25]:
for i, (name, group) in enumerate(regions.groupby(level=0)):
    #print(name, group)
    print(i, group.iloc[:, 1:3].head())

0                 region_id_3002 region_id_3003
                             0              0
3003 2017-12-07            0.0            1.0
     2017-12-08            0.0            1.0
     2017-12-09            0.0            1.0
     2017-12-10            0.0            1.0
     2017-12-11            0.0            1.0
1                 region_id_3002 region_id_3003
                             0              0
3006 2018-04-01            0.0            0.0
     2020-02-13            0.0            0.0
     2020-02-14            0.0            0.0
     2020-02-15            0.0            0.0
     2020-02-16            0.0            0.0
2                 region_id_3002 region_id_3003
                             0              0
3007 2017-12-07            0.0            0.0
     2017-12-08            0.0            0.0
     2017-12-09            0.0            0.0
     2017-12-10            0.0            0.0
     2017-12-11            0.0            0.0
3                 region_id_

# Make plots

I may need to normalize the input variables before I make the plots.